In [1]:
from itertools import combinations
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pymongo
import nltk
from nltk.corpus import stopwords
import pymorphy2
from tqdm import tqdm_notebook
nltk.download('stopwords')
stop = stopwords.words('russian')

from news.settings import MONGO_DB, MONGO_URI

[nltk_data] Downloading package stopwords to /home/user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
morph = pymorphy2.MorphAnalyzer()
def lemmatize(token):
    return morph.parse(token)[0].normal_form

In [3]:
client = pymongo.MongoClient(MONGO_URI)
db = client[MONGO_DB]
collection = db['news']

In [71]:
news = pd.DataFrame(list(collection.find()))
news.head()

,_id,text
0,5aa2affd1ca8146d0ab2cc9e,Россия может обратиться в суд ВТО из-за введен...
1,5aa2affd1ca8146d0ab2cc9f,В Пентагоне допустили возможность уничтожения ...
2,5aa2affd1ca8146d0c6e2103,Председатель Стратегического командования Воор...
3,5aa2affd1ca8146d0c6e2104,Еврокомиссия одобрила предложение предоставить...
4,5aa2affd1ca8146d0ab2cca0,Немецкий производитель одежды и обуви для спор...


In [79]:
news['text'] = news['text'].str.replace('\xa0', ' ').replace('\n', '')
news['text'] = news['text'].str.replace('\n', '')

In [101]:
tfidf = TfidfVectorizer(stop_words=stop, ngram_range=(1, 3), max_features=100000, preprocessor=lemmatize)

In [104]:
tfidf_matrix = tfidf.fit_transform(news['text']).toarray()

MemoryError: 

In [ ]:
duplicates = []
lengths = np.sum(tfidf_matrix ** 2, axis=1) ** 0.5
for i in tqdm_notebook(range(1, tfidf_matrix.shape[0])):
    sim = np.sum(tfidf_matrix * np.roll(tfidf_matrix, i, axis=0), axis=1) / (lengths * np.roll(lengths, i))
    indexes = np.argwhere(sim > 0.99)[:, 0]
    print(list(zip(indexes, indexes+i)))
    print(np.sum(sim > 0.99))
    break

In [103]:
import gc
gc.collect()

261

In [99]:
news.iloc[44517, 1]

' Пентагон не может подтвердить гибель сербских граждан при операции США в Ливии, заявил пресс-секретарь американского военного ведомства Питер Кук."Мы видели сообщения о том, что двое сербских заложников были убиты в Ливии. На данный момент у нас нет информации, свидетельствующей, что их смерть была результатом удара сил США, нанесенного против высокопоставленного лидера  и тренировочного лагеря ИГ в Ливии", — сказал Кук."Наши силы наблюдали за этим тренировочным лагерем на протяжении недель перед операцией, и в момент удара не было информации о присутствии гражданских лиц", — добавил он.Представитель Пентагона выразил соболезнования правительству Сербии и семьям погибших. "Мы будем делиться любой информацией с сербским правительством", — добавил он, заверив, что США стремятся не допустить гибели гражданских лиц во время операций за рубежом.В пятницу военные самолеты США нанесли авиаудары по ливийскому городу Сабрата, где погибли не менее 30 человек. Глава МИД Сербии Ивица Дачич сообщ

In [100]:
news.iloc[44518, 1]

' Россиянки Екатерина Петухова и Юлия Тимошинина на Кубке мира по прыжкам в воду в Рио-де-Жанейро завоевали олимпийские лицензии в прыжках с вышки.Петухова в предварительном раунде заняла 15-е место с результатом 294,15 балла. Тимошинина (270,45) стала 25-й и не вошла в число 18 спортсменов, которые должны были получить лицензии, но благодаря тому, что выше нее оказалось множество спортсменок с лицензиями, россиянка получила для сборной право выставить еще одну прыгунью на Олимпиаде.Финал в прыжках с вышки пройдет в воскресенье.'

In [ ]:
total = tfidf_matrix.shape[0] * (tfidf_matrix.shape[0] - 1) // 2
duplicates = [(i, j)
              for i, j in tqdm_notebook(combinations(range(tfidf_matrix.shape[0]), 2), total=total)
              if cosine_similarity(tfidf_matrix[i], tfidf_matrix[j])[0][0] > 0.8]